In [1]:
import sys
import random
sys.path.append("/pod/2/ke-lab/LUOZ/Singularity/iM6A")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
from Bio.SeqUtils import nt_search
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [6]:
paths = ('Test/mouseRAC10000_c{}.h5'.format(x) for x in range(1, 6)) # RRACH for mouse

In [7]:
models = [load_model(resource_filename('m6AAI', x), custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for x in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Read data

In [8]:
Data = pd.read_csv("/pod/2/ke-lab/LUOZ/m6AAI/Figure5/Fasta_CodonSwap.csv", keep_default_na=False)

In [9]:
Data

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,LastExonStart,LastExonEnd,m6AStart,...,Motif,m6APosition,m6ASequence,Protein,m6AToCodon,m6AInterval,Up,Mid,Down,POS
0,Gpr39,chr1,+,125676994,125873862,125677336,125872884,125872369,125873862,125872731,...,GAACT,362,GAACT,LIVVTLAVCWMPNQIRRIMAAAKPKHDWTRTYFRAYMILLPFSDTF...,360,0,CTGATTGTGGTGACGTTGGCCGTGTGTTGGATGCCCAATCAGATCC...,CCCCTCATCTTCCTAGCCTCCCGGCGCAGTAACTCTTCCTCCAGGA...,GAGGCTAAGCCCCAGCCCTTGAGTCCTGAGTCACCACAGACTGGCT...,195737
1,Plcl1,chr1,+,55405920,55754285,55406387,55751463,55751280,55754285,55751363,...,GGACT,83,GGACT,GQGDLLKNAKNEAVENIKQIQLACLSCGLSKGPGGGSEAKGKRSLE...,83,2,GGACAAGGAGACCTGCTGAAGAATGCCAAGAATGAA,GCCGTGGAGAACATAAAGCAAATCCAGCTGGCCTGCTTGTCTTGTG...,TTGGAAGCGATAGAGGAGAAAGAAAGTAGTGAGGAGAACGGGAAGC...,345443
2,Cdh20,chr1,+,104768528,104995481,104934096,104994385,104993879,104995481,104994314,...,GGACT,435,GGACT,LVLLILSMRRHRKQPYIIDDDENIHENIVRYDDEGGGEEDTEAFDI...,433,1,TTGGTGCTGCTCATCCTGTCCATGAGGCGACATCGCAAACAACCCT...,CTGCAGTCAGCCACCTCGGACTCAGAGCAGAGCTTTGATTTCCTCA...,GAGGGGCCCGCGCCCTTGTGGTGA,225786
3,Cdh7,chr1,+,109983736,110139001,109994179,110138355,110137861,110139001,110138138,...,AGACT,277,AGACT,LILLIVTMRRRKKEPLIFDEERDIRENIVRYDDEGGGEEDTEAFDM...,275,2,CTGATCCTCCTCATTGTCACTATGAGAAGACGAAAAAAAGAGCCTC...,AAAAACATCCCAGATAATGTCATTTTTAGGGAATTTATTTGGGAAA...,CAGACTTATGCATTTGAAGGAAATGGCTCAGTAGCTGAATCTCTCA...,154402
4,Cntnap5a,chr1,+,115684755,116587323,115685136,116580674,116580480,116587323,116580597,...,GAACT,117,GAACT,VIAVVTFITFCVIGIMTRFLYQHKQSHRTNQMKKEYPENLDNSFRN...,115,1,GTCATAGCAGTGGTGACATTTATTACTTTTTGTGTCATTGGAATCA...,CATAAGCAGTCACACCGCACCAATCAGATGAAGAAGGAATACCCAG...,ACTGAATGCAAACGAGAATATTTCATCTGA,895842
5,Brinp3,chr1,+,146494759,146902471,146514717,146902117,146901000,146902471,146901878,...,GGACT,878,GGACT,TSTYWLTRIQSFLYCNENGLLGSFSEETHSCTCPNDQVVCTAFLPC...,877,1,ACATCAACCTACTGGCTCACTCGCATCCAGTCTTTTCTCTACTGCA...,AGCATGCACTTTGACCCTGAAGCCATTCGGGACCTGATTTTGCAGC...,CTAGAGATAAGAGACCGTGTAAATAAACTCTCCCCACCCGGTCAGC...,407119
6,St18,chr1,+,6487230,6860934,6768897,6858600,6858459,6860934,6858532,...,GGACT,73,GGACT,GPINEQNFEAYVNTLTDMYSNLEQDYSPECKALLESIKQAVKGIHV_,73,1,GGGCCTATCAATGAGCAGAATTTT,GAAGCATATGTAAATACTCTCACAGACATGTACAGCAATCTGGAAC...,GTGAAGGGCATCCATGTGTAG,371302
7,Tfap2d,chr1,+,19103021,19166346,19103255,19166178,19165958,19166346,19166096,...,GGACA,138,GGACA,LITHGFGTPAICAALSTFQTVLSEMLNYLEKHTTHKNGGAADSGQG...,137,2,TTGATCACTCATGGTTTTGGGACACCAGCAATATGTGCAGCTCTAA...,AAACACACTACTCACAAAAACGGTGGTGCGGCAGACTCTGGCCAAG...,GTGAAAGAGGGCAAAACAGAAAAGACAGACTAG,63075
8,Dst,chr1,+,33908271,34307592,33908271,34307592,34307329,34307592,34307425,...,TGACA,96,TGACA,SRKTPSRPGSRAGSKAGSRASSRRGSDASDFDISEIQSVCSDVETV...,94,1,TCCAGGAAGACTCCCAGCCGCCCAGGGagcagagccggaagcaaa,gccggaagcagagccAGCAGCCGCCGAGGCAGCGACGCGTCCGACT...,CAGACACACAGACCTGTGCCCCGAGCAGGTTCTCGGCCATCCACAG...,399154
9,Ercc5,chr1,+,44147846,44181260,44148062,44181082,44180530,44181260,44180987,...,GGACA,457,GGACA,TQLRIDSFFRLAQQEKQDAKLIKSHRLNRAVTCILRKEREEKAPEL...,457,1,ACCCAGCTCCGGATTGATTCCTTTTTTAGACTAGCTCAGCAGGAAA...,GGGAGGCAAGGCTGTGTGTCCACTAGTAGCTCCAGCGAGGATGATG...,AAAAAACTGAAACTAAAGAGTATGAAGAGAAGGAAAAAGAAAACCTAA,33141


In [10]:
PreSequence = Data["PreSequence"].tolist()
Interval = Data["IntervalSequence"].tolist()
Up = Data["Up"].tolist()
Mid = Data["Mid"].tolist()
Down = Data["Down"].tolist()
Noncoding = Data["NoncodingSequence"].tolist()

In [11]:
Codon = { 
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M', 
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T', 
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K', 
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                  
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L', 
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P', 
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q', 
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R', 
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V', 
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A', 
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E', 
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G', 
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S', 
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L', 
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_', 
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W', 
    } 

In [12]:
Dict = dict(zip(range(32), [0]*32))
for key,value in Dict.items():
    motif = key
    Dict[key] = []

In [13]:
for i in range(len(Data)):
    
    pre = PreSequence[i]
    interval = Interval[i]

    up = Up[i]
    up = up.upper()
    mid = Mid[i]
    mid = mid.upper()
    down = Down[i]
    down =  down.upper()
    
    noncoding = Noncoding[i]
    
    for j in range(0,len(mid),3):
        
        Swap = {'R':['CGA', 'CGC', 'CGG', 'CGT', 'AGA', 'AGG'],
        'S':['TCA', 'TCC', 'TCG', 'TCT', 'AGC', 'AGT'],
        'L':['CTA', 'CTC', 'CTG', 'CTT', 'TTA', 'TTG'],
        'A':['GCA', 'GCC', 'GCG', 'GCT'],
        'G':['GGA', 'GGC', 'GGG', 'GGT'],
        'P':['CCA', 'CCC', 'CCG', 'CCT'],
        'T':['ACA', 'ACC', 'ACG', 'ACT'],
        'V':['GTA', 'GTC', 'GTG', 'GTT'],
        'I':['ATA', 'ATC', 'ATT'],
        '_':['TAA', 'TAG', 'TGA'],
        'C':['TGC', 'TGT'],
        'D':['GAC', 'GAT'],
        'E':['GAA', 'GAG'],
        'F':['TTC', 'TTT'],
        'H':['CAC', 'CAT'],
        'K':['AAA', 'AAG'],
        'N':['AAC', 'AAT'],
        'Q':['CAA', 'CAG'],
        'Y':['TAC', 'TAT'],
        'M':['ATG',"atg"],
        'W':['TGG',"tgg"]
        }
        
        codon = mid[j:j + 3]
        AA = Codon[codon]
        List = Swap[AA]
        List.remove(codon)
        
        for k in range(len(List)):
            seq = mid[0:j] + List[k] + mid[(j+3):]
            input_sequence = pre + interval + up + seq + down + noncoding
            x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
            y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
            m6AAI_prob = y[0, :, 1]
            value = m6AAI_prob[(Data.loc[i,"POS"])-1]
            Dvalue = value - Data.loc[i,"Probabilty"]
            Dict[(j/3)].append(Dvalue)
        
        
        
        


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


In [14]:
Count = pd.DataFrame(columns=["Score","Value"])
for key,value in Dict.items():
    Number = key
    Value = Dict[Number]
    Score = [Number]*len(Value)
    df = pd.DataFrame({"Score":Score, "Value":Value})
    Count = Count.append(df)


In [15]:
Count.to_csv("/pod/2/ke-lab/LUOZ/m6AAI/Figure5/Prob_CodonSwap.csv", index=0)